<a href="https://colab.research.google.com/github/tamim-18/ML-CookBook/blob/main/whisper_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [4]:
device

'cuda:0'

In [5]:
torch_dtype

torch.float16

In [6]:

model_id = "openai/whisper-large-v3"


In [7]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [9]:
pipe

In [12]:
audio = "/content/Top_5_English_Learning_Mistakes.mp3"

In [13]:
%%timeit
result = pipe(audio) # v2 gives better results if you don't provide a language.

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


10.8 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
result = pipe(audio)

In [17]:
result['text']

" Top five mistakes English learners make. What are the most common mistakes that English learners make? Which mistakes do most English learners need to correct in order to learn English much faster. Here are the top five English learning mistakes. One,using on grammar. This is the biggest, most common, and worst mistake. Research shows that grammar study, in fact, actually hurts English speaking ability. Why? Because English grammar is simply too complex to memorize and use logically. And real conversation is much too fast. You don't have enough time to think. Remember hundreds or thousands of grammar rules. Choose the correct one, then use it. Your logical left brain cannot do it. You must learn grammar intuitively and The English grammar and your brain gradually and automatically learns to use English grammar correctly. 2. Forcing speech. Both English students and teachers try to force speech before the learner is ready. The result is that most students speak English very slowly wit

In [18]:
result['chunks']

[{'timestamp': (0.0, 3.84),
  'text': ' Top five mistakes English learners make.'},
 {'timestamp': (5.02, 7.92),
  'text': ' What are the most common mistakes that English learners make?'},
 {'timestamp': (8.82, 17.46),
  'text': ' Which mistakes do most English learners need to correct in order to learn English much faster. Here are the'},
 {'timestamp': (17.46, 24.4),
  'text': ' top five English learning mistakes. One,using on grammar.'},
 {'timestamp': (24.4, 29.22),
  'text': ' This is the biggest, most common, and worst mistake.'},
 {'timestamp': (29.22, 36.68),
  'text': ' Research shows that grammar study, in fact, actually hurts English speaking ability.'},
 {'timestamp': (36.68, 38.32), 'text': ' Why?'},
 {'timestamp': (38.32, 43.66),
  'text': ' Because English grammar is simply too complex to memorize and use logically.'},
 {'timestamp': (44.44, 48.32),
  'text': ' And real conversation is much too fast.'},
 {'timestamp': (48.98, 50.94),
  'text': " You don't have enough ti